### convolutional conditional GAN

In [1]:
import pandas as pd
import numpy as np
import pyreadr as py # library to read .Rdata files in python
import os
import tensorflow as tf
from time import time
import pickle
import datetime
import random
import math
import csv
import time
import sys

In [2]:
from sklearn.metrics import multilabel_confusion_matrix,confusion_matrix,classification_report
from sklearn.preprocessing import StandardScaler,Normalizer
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV,KFold
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,f1_score,log_loss,recall_score,classification_report
from sklearn.preprocessing import scale, robust_scale, minmax_scale, maxabs_scale

In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset  
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

In [4]:
from src.models.model import AE_base
from src.data.common.dataset import FontDataset, PickledImageProvider

In [5]:
import torch
from torch.nn import functional as F
from torch.optim import SGD, Adam
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [6]:
batch_size = 8
validation_split = .15
test_split = .05
shuffle_dataset = True
random_seed = 42

lr = 0.0001

log_interval = 10
epochs = 200

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
from src.models.function import conv2d, deconv2d, batch_norm, lrelu, dropout
from src.data.common.dataset import NewFontDataset, PickledImageProvider, KoreanFontDataset

In [8]:
# get Dataset
data_dir = 'src/data/dataset/kor/'
train_set = KoreanFontDataset(PickledImageProvider(data_dir+'train.obj'), vector_size=20)
valid_set = KoreanFontDataset(PickledImageProvider(data_dir+'val.obj'), vector_size=20)
test_set = KoreanFontDataset(PickledImageProvider(data_dir+'test.obj'), vector_size=20)

processed 1000 examples
processed 2000 examples
processed 3000 examples
processed 4000 examples
processed 5000 examples
processed 6000 examples
processed 7000 examples
processed 8000 examples
processed 9000 examples
processed 10000 examples
processed 11000 examples
processed 12000 examples
processed 13000 examples
processed 14000 examples
processed 15000 examples
processed 16000 examples
processed 17000 examples
processed 18000 examples
processed 19000 examples
processed 20000 examples
processed 21000 examples
processed 22000 examples
processed 23000 examples
processed 24000 examples
processed 25000 examples
processed 26000 examples
processed 27000 examples
processed 28000 examples
processed 29000 examples
processed 30000 examples
processed 31000 examples
processed 32000 examples
processed 33000 examples
processed 34000 examples
processed 35000 examples
processed 36000 examples
processed 37000 examples
processed 38000 examples
processed 39000 examples
processed 40000 examples
processed

In [9]:
# get idx samplers
train_set_size = len(train_set)
valid_set_size = len(valid_set)
train_idxs = list(range(train_set_size))
valid_idxs = list(range(valid_set_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(train_idxs)
    np.random.shuffle(valid_idxs)

train_sampler = SubsetRandomSampler(train_idxs)
valid_sampler = SubsetRandomSampler(valid_idxs)

# get data_loaders
train_loader = DataLoader(train_set, 
                      batch_size=batch_size,
                      sampler=train_sampler
                      )
valid_loader = DataLoader(valid_set,
                        batch_size=batch_size,
                        sampler=valid_sampler
                        )
test_loader = DataLoader(test_set,
                        batch_size=len(test_set)
                        )

---

---
### discriminator

In [10]:
class Discriminator(nn.Module):
    def __init__(self, input_font_size = 128*128, img_dim=1, disc_dim = 64):
        super(Discriminator, self).__init__()
        self.conv1 = conv2d(img_dim, disc_dim, k_size=5, stride=2, pad=2, dilation=2, lrelu=False, bn=False)
        self.conv2 = conv2d(disc_dim, disc_dim*2, k_size=5, stride=4, pad=2, dilation=2)
        self.conv3 = conv2d(disc_dim*2, disc_dim*4, k_size=4, stride=4, pad=1, dilation=1)
        self.conv4 = conv2d(disc_dim*4, disc_dim*8)
        self.conv5 = conv2d(disc_dim*8, disc_dim*8)
        self.fc1 = nn.Linear(disc_dim*8 , 1)
        

    def forward(self, images, batch_size):
        batch_size = images.shape[0]
        images = images.unsqueeze(dim=1)
        #print(images.shape) # [8, 1, 128, 128]
        h1 = self.conv1(images)
        #print(h1.shape) # [8, 64, 64, 64]
        h2 = self.conv2(h1)
        #print(h2.shape) # [8, 128, 16, 16]
        h3 = self.conv3(h2)
        #print(h3.shape) # [8, 256, 4, 4]
        h4 = self.conv4(h3)        
        #print(h4.shape) # [8, 512, 2, 2]
        h5 = self.conv5(h4)        
        #print(h5.shape)
        out = torch.sigmoid(self.fc1(h5.reshape(batch_size, -1)))
        
        return out.squeeze()
    

---
### generator

In [21]:
embedded_dim = 148 ; gen_dim = 64
class Generator(nn.Module):
    def __init__(self, img_dim =1, embedded_dim = 148, conv_dim = 64):
        super(Generator, self).__init__()
        self.deconv1 = deconv2d(embedded_dim, conv_dim * 8, k_size=4, dilation=2, stride=2)
        self.deconv2 = deconv2d(conv_dim * 8, conv_dim * 4, k_size=4, dilation=2, stride=2)
        self.deconv3 = deconv2d(conv_dim * 4, conv_dim * 2, k_size=6, dilation=2, stride=4)
        self.deconv4 = deconv2d(conv_dim * 2, conv_dim * 1, k_size=6, dilation=2, stride=4)
        self.deconv5 = deconv2d(conv_dim * 1, img_dim, k_size=4, dilation=2, stride=2, bn=False)

        
    def forward(self, doc2vec_vector, gaussian_letter_vector, batch_size):
        input_cat = torch.cat((doc2vec_vector, gaussian_letter_vector),dim=1)
        embedded = input_cat.reshape([batch_size, embedded_dim, 1,1])   
        
        d1 = self.deconv1(embedded)
        #print(d1.shape)
        d2 = self.deconv2(d1)
        #print(d2.shape)
        d3 = self.deconv3(d2)
        #print(d3.shape)
        d4 = self.deconv4(d3)
        #print(d4.shape)
        d5 = self.deconv5(d4)
        #print(d5.shape)
        fake_target = d5.squeeze(dim=1)

        return fake_target
  

In [23]:
discriminator = Discriminator().cuda()
generator = Generator().cuda()

In [24]:
mse_criterion = nn.MSELoss().cuda()
bce_criterion = nn.BCELoss().cuda()
l1_criterion = nn.L1Loss().cuda

In [25]:
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=lr)

In [26]:
def train_discriminator(temp_batch_size, generator, discriminator, images, \
                        doc2vec_Vector, gaussian_letter_vector):
    d_optimizer.zero_grad()
    
    #train with the real image
    outputs = discriminator(images, batch_size)
    real_loss = F.binary_cross_entropy(outputs, Variable(torch.ones(batch_size)).cuda()) 
    real_score = outputs 
        
    #train with the fake image
    fake_images = generator(doc2vec_vector, gaussian_letter_vector, batch_size) 
    outputs = discriminator(fake_images, batch_size)    
    fake_loss = F.binary_cross_entropy(outputs, Variable(torch.zeros(batch_size)).cuda()) 
    fake_score = outputs 

    d_loss = real_loss + fake_loss 
    d_loss.backward() 
    d_optimizer.step()
    
    return d_loss, real_score, fake_score



In [27]:
def train_generator(temp_batch_size, gaussian_letter_vector, doc2vec_vector, \
                    images, generator, discriminator):

    g_optimizer.zero_grad()
    
    # Sample again for the generator and get output from discriminator
    fake_images = generator(doc2vec_vector, gaussian_letter_vector, batch_size) 
    outputs = discriminator(fake_images, batch_size)

    g_l1_loss= F.l1_loss(fake_images, images)
    g_bce_loss = F.binary_cross_entropy(outputs, Variable(torch.ones(batch_size)).cuda())
    
    g_loss = g_l1_loss + g_bce_loss
    g_loss.backward()
    g_optimizer.step()
    
    return g_loss, g_l1_loss




In [28]:
def valid_discriminator(temp_batch_size, generator, discriminator, images, \
                        doc2vec_Vector, gaussian_letter_vector):
    #train with the real image
    outputs = discriminator(images, batch_size)
    real_loss = F.binary_cross_entropy(outputs, Variable(torch.ones(batch_size)).cuda()) 
    real_score = outputs 
        
    #train with the fake image
    fake_images = generator(doc2vec_vector, gaussian_letter_vector, batch_size) 
    outputs = discriminator(fake_images, batch_size)    
    fake_loss = F.binary_cross_entropy(outputs, Variable(torch.zeros(batch_size)).cuda()) 
    fake_score = outputs 

    d_loss = real_loss + fake_loss 
    
    return d_loss, real_score, fake_score



In [29]:
def valid_generator(temp_batch_size, gaussian_letter_vector, doc2vec_vector, \
                    images, generator, discriminator):
    
    # Sample again for the generator and get output from discriminator
    fake_images = generator(doc2vec_vector, gaussian_letter_vector, batch_size) 
    outputs = discriminator(fake_images, batch_size)

    g_l1_loss= F.l1_loss(fake_images, images)
    g_bce_loss = F.binary_cross_entropy(outputs, Variable(torch.ones(batch_size)).cuda())
    
    g_loss = g_l1_loss + g_bce_loss
    
    return g_loss, g_l1_loss




In [30]:
np.random.seed(seed=304)
random_table = np.random.normal(size=[2350,128])

In [31]:
# set number of epochs and initialize figure counter
num_epochs = 100
num_batches = len(train_loader)
num_fig = 0

final_g_total_loss = []
final_g_l1_loss = []
final_d_loss = []

final_g_total_loss_v = []
final_g_l1_loss_v = []
final_d_loss_v = []

for epoch in range(num_epochs):
    temp_g_total_loss = []
    temp_g_l1_loss = []
    temp_d_loss = []
    
    temp_g_total_loss_v = []
    temp_g_l1_loss_v = []
    temp_d_loss_v = []

    if epoch % 1 == 0:
        print('----------------------------%d epoch----------------------------------'%(epoch+1))
    
    for n, (vectors, font) in enumerate(train_loader):
        doc2vec_vector = vectors['font_doc2vec']
        letter_vector = vectors['word_index']
        gaussian_letter_vector = random_table[letter_vector] 

        images = font.float().to(device)
        doc2vec_vector = doc2vec_vector.float().to(device)
        gaussian_letter_vector = torch.from_numpy(gaussian_letter_vector).float().to(device)
        temp_batch_size = letter_vector.size()[0]
    
        # Train the discriminator
        d_loss, real_score, fake_score =train_discriminator(temp_batch_size, generator, discriminator, \
                                                            images, doc2vec_vector, gaussian_letter_vector)
       
        
        # Train the generator
        g_loss, g_l1_loss = train_generator(temp_batch_size, gaussian_letter_vector, doc2vec_vector, \
                                            images, generator, discriminator)
                 
        temp_g_total_loss.append(g_loss.item())
        temp_g_l1_loss.append(g_l1_loss.item())
        temp_d_loss.append(d_loss.item())
        

        if n % 1000 == 0:
            final_g_total_loss.append(np.mean(temp_g_total_loss))
            final_g_l1_loss.append(np.mean(temp_g_l1_loss))
            final_d_loss.append(np.mean(temp_d_loss))       

            print('Epoch [%d/%d], Step[%d/%d], d_loss: %.4f, g_loss: %.4f, ' 
                  'D(x): %.2f, D(G(z)): %.2f' 
                  %(epoch + 1, num_epochs, n+1, num_batches, np.mean(final_d_loss),np.mean(final_g_total_loss),
                    real_score.data.mean(), fake_score.data.mean()))
        
    # validation part
    for n, (vectors, font) in enumerate(valid_loader):
        
        doc2vec_vector = vectors['font_doc2vec']
        letter_vector = vectors['word_index']
        gaussian_letter_vector = random_table[letter_vector] 

        images = font.float().to(device)
        doc2vec_vector = doc2vec_vector.float().to(device)
        gaussian_letter_vector = torch.from_numpy(gaussian_letter_vector).float().to(device)
        temp_batch_size = letter_vector.size()[0]
        
        # Train the discriminator
        d_loss, real_score, fake_score =valid_discriminator(temp_batch_size, generator, discriminator, \
                                                            images, doc2vec_vector, gaussian_letter_vector)
       
        
        # Train the generator
        g_loss, g_l1_loss = valid_generator(temp_batch_size, gaussian_letter_vector, doc2vec_vector, \
                                            images, generator, discriminator)
        
        temp_g_total_loss_v.append(g_loss.item())
        temp_g_l1_loss_v.append(g_l1_loss.item())
        temp_d_loss_v.append(d_loss.item())



        if n % 1000 == 0:

            final_g_total_loss_v.append(np.mean(temp_g_total_loss_v))
            final_g_l1_loss_v.append(np.mean(temp_g_l1_loss_v))
            final_d_loss_v.append(np.mean(temp_d_loss_v))       

            print('Epoch [%d/%d], Step[%d/%d], valid_d_loss: %.4f, valid_g_loss: %.4f, ' 
                  'D(x): %.2f, D(G(z)): %.2f' 
                  %(epoch + 1, num_epochs, n+1, num_batches, np.mean(final_d_loss_v),np.mean(final_g_total_loss_v),
                    real_score.data.mean(), fake_score.data.mean()))

        


----------------------------1 epoch----------------------------------
Epoch [1/100], Step[1/25263], d_loss: 1.4083, g_loss: 1.9716, D(x): 0.51, D(G(z)): 0.51
Epoch [1/100], Step[1001/25263], d_loss: 1.3998, g_loss: 1.4130, D(x): 0.50, D(G(z)): 0.50
Epoch [1/100], Step[2001/25263], d_loss: 1.3962, g_loss: 1.2108, D(x): 0.50, D(G(z)): 0.50
Epoch [1/100], Step[3001/25263], d_loss: 1.3941, g_loss: 1.1057, D(x): 0.50, D(G(z)): 0.50
Epoch [1/100], Step[4001/25263], d_loss: 1.3928, g_loss: 1.0411, D(x): 0.50, D(G(z)): 0.50
Epoch [1/100], Step[5001/25263], d_loss: 1.3919, g_loss: 0.9971, D(x): 0.50, D(G(z)): 0.50
Epoch [1/100], Step[6001/25263], d_loss: 1.3912, g_loss: 0.9653, D(x): 0.50, D(G(z)): 0.50


KeyboardInterrupt: 

---

In [ ]:
#plt.plot(final_g_total_loss)
plt.plot(final_g_l1_loss)
plt.plot(final_g_l1_loss_v)
#plt.plot(final_d_loss)
plt.show()

In [ ]:
# save
savePath = "D:/투빅스 2019/generator_1210.pth"
torch.save(generator.state_dict(), savePath)



In [ ]:
# save
savePath = "D:/투빅스 2019/discriminator_1210.pth"
torch.save(discriminator.state_dict(), savePath)



In [ ]:
# load
#new_model = TestModel()
#new_model.load_state_dict(torch.load("./output/test_model.pth"))


---

In [ ]:
for i, (vectors, font) in enumerate(test_loader):

    doc2vec_vector = vectors['font_doc2vec']
    letter_vector = vectors['word_index']

    images, letter_vector = font.float().to(device), letter_vector.float().to(device)
    doc2vec_vector = doc2vec_vector.float().to(device)

    temp_batch_size = letter_vector.size()[0]

    fake_images = generator(images, doc2vec_vector, letter_vector, temp_batch_size) 
    if i ==0: break

In [ ]:
fake_images_view = fake_images.view(fake_images.size(0), 128, 128)
images_view = images.view(images.size(0), 128, 128)

plt.figure(figsize=(10, 200))
for i in range(8):
    plt.subplot(50, 4, i*2+1)
    plt.imshow(images_view[i].cpu().detach().numpy())
    plt.subplot(50, 4, (i+1)*2)
    plt.imshow(fake_images_view[i].cpu().detach().numpy())
    